In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA, ARMA
from statsmodels.tsa.stattools import acf, pacf

import xgboost as xgb
from xgboost import plot_importance
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings

import datetime
warnings.filterwarnings('ignore')
path = "D:/program/program1/Cargo_forecast/dot_data_all/"
print(os.listdir(path))

['20181207dot1.xlsx', '20181207_莉娜_点部#1.xlsx', '20181207_莉娜_点部#2.xlsx', '20181207_莉娜_点部#3.xlsx', '20190103_1.csv', '20190103_2.csv', '20190103_点部取货量_1.xlsx', '20190103_点部取货量_2.xlsx', '20190103_点部派货量_1.xlsx', '20190103_点部派货量_2.xlsx', '20190218.csv', 'accurate.xlsx', 'comments.txt', 'dot.csv', 'dot_result.csv', 'dot_result_new.csv', 'feature_importance.png', 'rmse1.csv', 'test.csv', 'test_predict.png']


In [123]:
dot_data_1 = pd.read_csv(path + '20190218.csv', header = None, names = ['dot', 'order_date', 'order_num'])
dot_data_2 = pd.read_csv(path + 'dot.csv', header = None, names = ['dot', 'dot_type', 'area', 'city_code', 'department', 'province', 'city', 'district', 'town', 'village'])


In [124]:
dot_data_1['order_date'] = pd.to_datetime(dot_data_1['order_date'])
dot_data_1 = dot_data_1[dot_data_1['order_date'] <= pd.to_datetime('2019-01-31')]
dot_data_1.tail()

,dot,order_date,order_num
1117985,凤岗鲤牙塘点部,2019-01-31,4
1117986,南宁西乡塘点部,2019-01-31,1
1117987,双流颐康点部,2019-01-31,1
1117988,德阳古城点部,2019-01-31,2
1117989,贵阳机场点部,2019-01-31,2


In [125]:
dot_data_2.loc[dot_data_2['city'].isnull(), ['city']] = dot_data_2.loc[dot_data_2['city'].isnull(), ['province']].values
dot_data_2.loc[dot_data_2['district'].isnull(), ['district']] = dot_data_2.loc[dot_data_2['district'].isnull(), ['city']].values
dot_data_2.head()

,dot,dot_type,area,city_code,department,province,city,district,town,village
0,清江新下塘点部,大点部,华东,577,清江操作部,浙江省,温州市,乐清市,城东街道,新下塘村
1,香洲洪湾点部,大点部,华南,756,香洲操作部,广东省,珠海市,香洲区,南屏镇,洪湾社区
2,贵阳三甫点部,大点部,华北,851,贵阳操作部,贵州省,贵阳市,观山湖区,金华镇,三甫村
3,镇江大港点部,大点部,华东,511,镇江操作部,江苏省,镇江市,镇江新区,大路镇,大路村
4,三水竹山点部,大点部,华南,757,三水操作部,广东省,佛山市,三水区,乐平镇,竹山村


In [126]:
dot_data = pd.merge(dot_data_1, dot_data_2, on = ['dot'], how = 'right')
dot_data.sort_values(by = 'order_date', ascending = True, inplace = True)
dot_data.dropna(axis = 0, inplace = True)
dot_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 724863 entries, 0 to 724862
Data columns (total 12 columns):
dot           724863 non-null object
order_date    724863 non-null datetime64[ns]
order_num     724863 non-null float64
dot_type      724863 non-null object
area          724863 non-null object
city_code     724863 non-null int64
department    724863 non-null object
province      724863 non-null object
city          724863 non-null object
district      724863 non-null object
town          724863 non-null object
village       724863 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 71.9+ MB


In [129]:
dot_data.head(10)

,dot,order_date,order_num,dot_type,area,city_code,department,province,city,district,town,village
0,南汇临港点部,2017-08-01,4.0,大点部,华东,21,南汇操作部,上海,上海市,浦东新区,泥城镇,云帆苑社区
72822,嘉定博学点部,2017-08-01,1.0,大点部,华东,21,嘉定操作部,上海,上海市,嘉定区,马陆镇,彭赵村
56531,宁波机场点部,2017-08-01,1.0,航空组,华东,574,宁波操作部,浙江省,宁波市,海曙区,古林镇,共任村
165727,龙华华宁点部,2017-08-01,1.0,大点部,华南,755,龙华操作部,广东省,深圳市,龙华区,大浪街道,上横朗社区华宁路
165186,福田泰然点部,2017-08-01,1.0,大点部,华南,755,福田操作部,广东省,深圳市,福田区,沙头街道,泰然社区
96820,相城渭塘点部,2017-08-01,2.0,大点部,华东,512,相城操作部,江苏省,苏州市,相城区,渭塘镇,骑河村
17258,绍兴湖塘点部,2017-08-01,1.0,大点部,华东,575,绍兴操作部,浙江省,绍兴市,柯桥区,钱清镇,钱清村
45048,新区马涧点部,2017-08-01,3.0,大点部,华东,512,新区操作部,江苏省,苏州市,虎丘区,枫桥街道,白马涧社区第二居委会
26302,普陀遵义点部,2017-08-01,2.0,大点部,华东,21,普陀操作部,上海,上海市,静安区,江宁路街道,三星坊居委会
96226,太仓沙溪点部,2017-08-01,1.0,大点部,华东,512,太仓操作部,江苏省,苏州市,太仓市,沙溪镇,松南村


In [104]:
dot_data.dot_type.unique()

array(['大点部', '小点部', '航空组', '虚拟点部', '二级中转场'], dtype=object)

In [133]:
pd.concat([dot_data, pd.get_dummies(dot_data['dot_type']), pd.get_dummies(dot_data['area']), pd.get_dummies(dot_data['province'])],axis = 1).head()

,dot,order_date,order_num,dot_type,area,city_code,department,province,city,district,...,海南省,湖北省,湖南省,甘肃省,福建省,贵州省,辽宁省,重庆,陕西省,黑龙江省
0,南汇临港点部,2017-08-01,4.0,大点部,华东,21,南汇操作部,上海,上海市,浦东新区,...,0,0,0,0,0,0,0,0,0,0
72822,嘉定博学点部,2017-08-01,1.0,大点部,华东,21,嘉定操作部,上海,上海市,嘉定区,...,0,0,0,0,0,0,0,0,0,0
56531,宁波机场点部,2017-08-01,1.0,航空组,华东,574,宁波操作部,浙江省,宁波市,海曙区,...,0,0,0,0,0,0,0,0,0,0
165727,龙华华宁点部,2017-08-01,1.0,大点部,华南,755,龙华操作部,广东省,深圳市,龙华区,...,0,0,0,0,0,0,0,0,0,0
165186,福田泰然点部,2017-08-01,1.0,大点部,华南,755,福田操作部,广东省,深圳市,福田区,...,0,0,0,0,0,0,0,0,0,0
